Import modules

In [80]:
import pandas as pd
import numpy as np
import os

Declare constants and import the dataset

In [81]:
DIR = '../dataset/'
OUT_DIR = '../udataset/acs'
DATASET = 'acs_5yr_est_selected_economic_characteristics_2010-2022.csv'
df = pd.read_csv(DIR + DATASET)

Rename columns and remove missing values

In [82]:
df.columns = ['label', 'category', 'state', 'estimate', 'moe', 'percent', 'percent moe', 'year']
df = df.replace("(X)", "")

Convert estimate to numeric

In [83]:
df['estimate'] = pd.to_numeric(df['estimate'].str.replace(',', ''), errors='coerce')

Convert margin or error columns to numeric

In [84]:
df['moe'] = pd.to_numeric(df['moe'].str.replace('±', '').str.replace(',', ''), errors='coerce')
df['percent moe'] = pd.to_numeric(df['percent moe'].str.replace('±', '').str.replace(',', ''), errors='coerce')

Clean the percent column by removing percentage signs and replacing any non-percentages with 100%

In [85]:
def filter_percent(val):
    if pd.isnull(val):
        return val
    if ',' in val and '%' not in val:
        return 100
    return val.replace('%', '')

df['percent'] = df['percent'].apply(filter_percent)
df['percent'] = pd.to_numeric(df['percent'], errors='coerce')

Preview the cleaned dataset

In [86]:
df

,label,category,state,estimate,moe,percent,percent moe,year
0,EMPLOYMENT STATUS,Header,Alabama,NaN,NaN,NaN,NaN,2010
1,Population 16 years and over,EMPLOYMENT STATUS,Alabama,3714504.0,1588.0,100.0,NaN,2010
2,In labor force,EMPLOYMENT STATUS,Alabama,2246848.0,5576.0,60.5,0.1,2010
3,Civilian labor force,EMPLOYMENT STATUS,Alabama,2230879.0,5531.0,60.1,0.1,2010
4,Employed,EMPLOYMENT STATUS,Alabama,2036867.0,5904.0,54.8,0.2,2010
...,...,...,...,...,...,...,...,...
98015,18 years and over,PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME...,Puerto Rico,NaN,NaN,39.7,0.4,2021
98016,18 to 64 years,PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME...,Puerto Rico,NaN,NaN,40.1,0.4,2021
98017,65 years and over,PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME...,Puerto Rico,NaN,NaN,38.5,0.5,2021
98018,People in families,PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME...,Puerto Rico,NaN,NaN,40.0,0.4,2021


Look at the dtypes of the columns

In [87]:
df.dtypes

label           object
category        object
state           object
estimate       float64
moe            float64
percent        float64
percent moe    float64
year             int64
dtype: object

Group the data by category of question asked, sort it chronologically, and output it

In [89]:
os.makedirs(OUT_DIR, exist_ok=True)

grouping: dict[str,list] = {}
current = None

income_and_benefits = 'INCOME AND BENEFITS'
poverty_line = 'POVERTY LEVEL'

for _, row in df.iterrows():
    cat: str = row['category']
    label: str = row['label']

    if label.startswith(income_and_benefits):
        label = income_and_benefits
        
    if label.endswith(poverty_line):
        label = 'PERCENT POVERTY LINE'

    if cat == 'Header':
        if not label in grouping:
            grouping[label] = []
        current = label
    else:
        grouping[current].append(row)

for label, data in grouping.items():
    df_label = pd.DataFrame(data)
    df_label = df_label.drop(columns=['category'])
    df_label = df_label.sort_values(by=["year","state"], ignore_index=True)
    df_label.to_csv(f"../udataset/acs/{label}.csv", index=False)